In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

### prep Data

In [2]:
europe_or_nearby = ['Austria', 'Belarus', 'Belgium', 'Bulgaria', 'Croatia', 'Czechia',
       'Denmark', 'Estonia', 'Finland', 'France', 'Germany', 'Greece',
       'Hungary', 'Ireland', 'Italy', 'Luxembourg', 'Netherlands',
       'North Macedonia', 'Norway', 'Poland', 'Portugal', 'Romania',
       'Slovakia', 'Slovenia', 'Spain', 'Sweden', 'Switzerland',
       'United Kingdom']

In [3]:
distance = pd.read_csv('data/controls/distance.csv', index_col=0)
subsidies = pd.read_csv('data/controls/energy_subsidies.csv', index_col=0)
urss = pd.read_csv('data/controls/ex_urss_influenced.csv', index_col=0)
gdppc = pd.read_csv('data/controls/GDPperCap.csv', index_col=0)
unemployment = pd.read_csv('data/controls/unemployment.csv', index_col=0)

In [4]:
# keep change in subsidies from 2021 to 2022
subsidies['increase'] = subsidies['2022'] - subsidies['2021']
subsidies = subsidies[['increase']].rename(columns={'increase': 'energy_subsidies_increase'})

In [5]:
# will talke 2022 (not the change), to account for country economic disparities

# keep only countries in Europe or nearby for gdppc
gdppc = gdppc.loc[gdppc.index.isin(europe_or_nearby)]
gdppc = gdppc[['2022']].rename(columns={'2022': 'gdppc'})

In [6]:
unemployment = unemployment.loc[unemployment.index.isin(europe_or_nearby)]
unemployment['increase'] = unemployment['2022'] - unemployment['2021']
unemployment = unemployment[['2022']].rename(columns={'2022': 'unemployment_increase'})

In [7]:
bartik = pd.read_csv('data/FINAL_BARTIK.csv', index_col=0)

In [8]:
approvals = pd.read_csv('../Design/data/Approvals_cleaned.csv')
approvals.drop(columns=['Sweden', 'Italy'], inplace=True)

def compute_approval_change(start_date, end_date):
    df = approvals.copy()
    df['Date'] = pd.to_datetime(df['Date'])  # ensure datetime format

    # Convert wide to long format
    df_long = df.melt(id_vars='Date', var_name='country', value_name='approval')

    # Filter date range
    df_long = df_long[(df_long['Date'] >= start_date) & (df_long['Date'] <= end_date)]

    # Calculate change from start to end per country
    start_approvals = df_long[df_long['Date'] == pd.to_datetime(start_date)]
    end_approvals = df_long[df_long['Date'] == pd.to_datetime(end_date)]

    merged = pd.merge(start_approvals, end_approvals, on='country', suffixes=('_start', '_end'))
    merged['approval_change'] = merged['approval_end'] - merged['approval_start']

    return merged[['country', 'approval_change']]

approval_change = compute_approval_change('2022-06-01', '2022-12-01').dropna()

In [9]:
approval_change.index = approval_change['country']
approval_change = approval_change[['approval_change']]

### Merge all together

In [10]:
# merge all controls into one DataFrame
dataset = distance.join(subsidies).join(gdppc).join(unemployment).join(bartik).join(approval_change) # . join(urss)

In [11]:
dataset.dropna(inplace=True)

In [12]:
# normalize the controls exept for the approval change and ex_urss_influenced
for col in dataset.columns:
    if col not in ['approval_change']:
        dataset[col] = (dataset[col] - dataset[col].mean()) / dataset[col].std()

dataset

,distance_to_kyiv_km,energy_subsidies_increase,gdppc,unemployment_increase,bartik_iv,approval_change
country,,,,,,
Austria,-0.723659,-0.341913,0.146928,-0.156204,-0.408905,-3.0
Belgium,0.427343,-0.688853,0.105088,0.060580,-0.083559,-5.0
Bulgaria,-0.773786,1.277144,-1.030335,-0.427747,0.632631,-8.0
Croatia,-0.519050,0.005028,-0.883014,0.581913,-0.125074,1.0
Czechia,-0.593214,-0.804500,-0.583107,-1.194370,1.326173,-6.0
Denmark,-0.320450,-0.688853,0.633161,-0.365488,-0.852492,-3.0
Estonia,-0.710322,-0.226266,-0.577946,0.060955,1.461019,-2.0
Finland,-0.604646,-0.688853,0.093822,0.491524,-0.694479,0.0
France,0.702745,0.351969,-0.192017,0.712434,-0.927250,-6.0


In [13]:
dataset.head(2)

,distance_to_kyiv_km,energy_subsidies_increase,gdppc,unemployment_increase,bartik_iv,approval_change
country,,,,,,
Austria,-0.723659,-0.341913,0.146928,-0.156204,-0.408905,-3.0
Belgium,0.427343,-0.688853,0.105088,0.060580,-0.083559,-5.0


In [17]:
# regress approval_change on pca_controls and bartik
import statsmodels.api as sm
X = dataset[['bartik_iv', 'gdppc']]
X = sm.add_constant(X)  # add constant term for intercept
model = sm.OLS(dataset['approval_change'], X).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:        approval_change   R-squared:                       0.177
Model:                            OLS   Adj. R-squared:                  0.094
Method:                 Least Squares   F-statistic:                     2.145
Date:                Tue, 24 Jun 2025   Prob (F-statistic):              0.143
Time:                        11:14:45   Log-Likelihood:                -64.460
No. Observations:                  23   AIC:                             134.9
Df Residuals:                      20   BIC:                             138.3
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -3.1304      0.892     -3.509      0.0